In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [2]:
def getAveragePer1000(filepath,n,filename):
    csv_data = pd.read_csv(filepath, low_memory = False)#防止弹出警告
    csv_df = pd.DataFrame(csv_data)
    averages = [np.mean(csv_df.loc[i:i+n-1, "P"]) for i in range(0, csv_df.shape[0], n)]
    np.savetxt('../csvfile_bin_1000_AveragewithoutX/'+filename+'.csv', averages, delimiter = ',')
    return averages

In [3]:
def ifExistCSVFile_1000Average(filename):
    filepath='../csvfile_bin_1000_AveragewithoutX/'
    files=os.listdir(filepath)
    if filename in files:
        return True
    else:
        return False

In [4]:
arrforDistribution=[]
for curDir, dirs, files in os.walk(top="../csvfiles/"):
    for file in files:
        if file.startswith("index_prob_split_"):
            fileprefix=file[file.index("chr"):file.index(".")]
            if fileprefix.split("_")[0]!="chrX":
                filepath=os.path.join(curDir,file)
                arrforDistribution+=getAveragePer1000(filepath=filepath,n=1000,filename=fileprefix)
        

In [5]:
mhc_1000=getAveragePer1000(filename="mhc",filepath="../csvfiles/mhc.csv",n=1000)
arrforDistribution+=mhc_1000

In [6]:
#get mean of Distribution
mean_arr=np.mean(arrforDistribution)
#get standard deviation of Distribution
std_arr=np.std(arrforDistribution)
print("mean_arr",mean_arr)
print("std_arr",std_arr)

mean_arr -0.019077723390066942
std_arr 0.006128439002438663


In [7]:
def calculatePos_PforMHC(mean_arr,std_arr):
    result=dict()
    vcf_in=vcf("../mhcdataset/1kgp.29720000-33130000.vcf","r")
    df=pd.read_csv("../csvfiles/posmhc.csv")
    cal=df["P"].to_list()
    pos=[]
    for rec in vcf_in.fetch():
        pos.append(rec.pos)
    pos=sorted(list(set(pos)))
    for i in range(len(cal)):
        mid_res=stats.norm.cdf(cal[i], mean_arr, std_arr)
        result[pos[i]]=mid_res
    return pos,result

In [8]:
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):
    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../csvfiles_compare_with_0.01pvaluewithoutX/"+filename+".csv", arr, delimiter = ',')
    return filename

In [9]:
for curDir, dirs, files in os.walk(top="../csvfile_bin_1000_AveragewithoutX/"):
    for file in files:            
            filename=file[:file.index(".")]
            prefix=filename.split("_")[0]
            if prefix!='chrX':
                filepath=os.path.join(curDir,file)
                compareWith0_01Pvalue(mean_arr=mean_arr,std_arr=std_arr,filename=filename,filepath=filepath)